In [8]:
pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 100.0/100.0 kB 191.5 kB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [68]:
import pandas as pd
from fuzzywuzzy import fuzz
import jieba
import os
import re

In [58]:
# Define root directory
root_dir = r"D:/铁书宁/2025 HKU predoc/RA task1_20250405"

df_path = os.path.join(root_dir, "ar-sale-buyer-match_forShuning.csv")
df = pd.read_csv(df_path)

### Step 1: City match for the AR buyer and customer.

In [59]:
# Import region dataset
city_path = os.path.join(root_dir, "China-City-List-latest.csv")
city_df = pd.read_csv(city_path, encoding = 'utf-8', header = 1)

# Keep city names
city_df = city_df[['Adm1_Name_ZH','Adm2_Name_ZH']]
city_df = city_df.drop_duplicates()

# Delele the word "省" and "市"
city_df['Adm1_Name_ZH'] = city_df['Adm1_Name_ZH'].str[:-1]
city_df['Adm2_Name_ZH'] = city_df['Adm2_Name_ZH'].str[:-1]

# Generate a region list
regions = pd.concat([city_df['Adm1_Name_ZH'], city_df['Adm2_Name_ZH']])
regions = list(regions.drop_duplicates())

In [60]:
# Define a region extract function
def extract_region(name):
    for region in regions:
        if region in name:
            return region
    return None

df['ar_buyer_region'] = df['ar_buyer_standard'].astype(str).apply(extract_region)
df['cus_region'] = df['cus_name_standard'].astype(str).apply(extract_region)

In [61]:
# City match for ar_buyer and cus_name, also keep the N.A.
df_citymatch = df[
    (df['ar_buyer_region'] == df['cus_region']) |
    (df['ar_buyer_region'].isna() & df['cus_region'].isna())
]

In [62]:
df_citymatch.head(10)

,wind_code,sec_name,ar_by_buyer_name,ar_buyer_standard,ar_buyer_subid,cus_name,cus_name_standard,cus_name_subid,ar_buyer_region,cus_region
77,000004.SZ,国华网安,内蒙古公安厅,内蒙古公安厅,6,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
107,000004.SZ,国华网安,周文亮,周文亮,8,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
121,000004.SZ,国华网安,四川科伦医药贸易有限公司,四川科伦医药贸易有限公司,9,四川源亨众生医药有限公司,四川源亨众生医药有限公司,2,四川,四川
136,000004.SZ,国华网安,国药控股四川医药股份有限公司,国药控股四川医药股份有限公司,10,四川源亨众生医药有限公司,四川源亨众生医药有限公司,2,四川,四川
152,000004.SZ,国华网安,宁夏耀康医药有限公司,宁夏耀康医药有限公司,11,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
183,000004.SZ,国华网安,山东瑞康医药股份有限公司,山东瑞康医药股份有限公司,13,山东众智医药有限公司,山东众智医药有限公司,4,山东,山东
198,000004.SZ,国华网安,山东豪诺医药有限公司,山东豪诺医药有限公司,14,山东众智医药有限公司,山东众智医药有限公司,4,山东,山东
302,000004.SZ,国华网安,杜秀珍,杜秀珍,21,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
317,000004.SZ,国华网安,"杨秀兰,丁亮","杨秀兰,丁亮",22,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
351,000004.SZ,国华网安,河北龙海新药经营有限公司,河北龙海新药经营有限公司,24,河北启源医药有限公司,河北启源医药有限公司,7,河北,河北


### Step 2: Use Jieba to seperate the string to words.

In [63]:
# Use jieba to seperate the words.
df_citymatch = df_citymatch.copy()
df_citymatch['ar_buyer_seg'] = df_citymatch['ar_buyer_standard'].astype(str).apply(lambda x: jieba.lcut(x))
df_citymatch['cus_name_seg'] = df_citymatch['cus_name_standard'].astype(str).apply(lambda x: jieba.lcut(x))

In [64]:
df_citymatch = df_citymatch.copy()
df_citymatch['cus_name_seg_str'] = df_citymatch['cus_name_seg'].apply(lambda x: ' '.join(x))
df_citymatch['ar_buyer_seg_str'] = df_citymatch['ar_buyer_seg'].apply(lambda x: ' '.join(x))

#### Jieba just for Chinese names, so for English names, we normalize the text by removing spaces and punctuation.

In [65]:
# Determine if it is English name
def is_english(text):
    return bool(re.match(r'^[\x00-\x7F]+$', text))

# Retain letters and numbers, remove special symbols such as spaces, punctuation, etc., and convert to lower case
def clean_english_text(text):
    return re.sub(r'[^a-zA-Z0-9]', '', text).lower()

In [69]:
df_citymatch['ar_buyer_seg_str'] = df_citymatch['ar_buyer_seg_str'].astype(str).apply(
    lambda x: clean_english_text(x) if is_english(x) else x
)

df_citymatch['cus_name_seg_str'] = df_citymatch['cus_name_seg_str'].astype(str).apply(
    lambda x: clean_english_text(x) if is_english(x) else x
)

### Step 3: Compute fuzzy match score

In [70]:
df_fuzzymatch = df_citymatch.iloc[:,:]

# Fuzzy match function
def fuzzy_match(name1, name2):
    return fuzz.token_sort_ratio(name1, name2)

# Apply fuzzy matching to your columns
df_fuzzymatch['match_score'] = df_fuzzymatch.apply(lambda row: fuzzy_match(row['cus_name_seg_str'], row['ar_buyer_seg_str']), axis=1)

In [79]:
output_path1 = os.path.join(root_dir, "df_fuzzymatch_v2.csv")
df_fuzzymatch.to_csv(output_path1, encoding = 'utf-8-sig')

#### (1) Choose 85 as a cutoff score

In [71]:
df_fuzzymatch85 = df_fuzzymatch[df_fuzzymatch['match_score'] >= 85]
df_fuzzymatch85.head(10)

,wind_code,sec_name,ar_by_buyer_name,ar_buyer_standard,ar_buyer_subid,cus_name,cus_name_standard,cus_name_subid,ar_buyer_region,cus_region,ar_buyer_seg,cus_name_seg,cus_name_seg_str,ar_buyer_seg_str,match_score
430,000004.SZ,国华网安,福建乐游网络科技有限公司,福建乐游网络科技有限公司,29,福建乐游网络科技有限公司,福建乐游网络科技有限公司,11,福建,福建,"[福建, 乐游, 网络科技, 有限公司]","[福建, 乐游, 网络科技, 有限公司]",福建 乐游 网络科技 有限公司,福建 乐游 网络科技 有限公司,100
432,000004.SZ,国华网安,福建乐游网络科技有限公司,福建乐游网络科技有限公司,29,福建摩格网络科技有限公司,福建摩格网络科技有限公司,13,福建,福建,"[福建, 乐游, 网络科技, 有限公司]","[福建, 摩格, 网络科技, 有限公司]",福建 摩格 网络科技 有限公司,福建 乐游 网络科技 有限公司,87
446,000004.SZ,国华网安,福建众邦药业有限公司,福建众邦药业有限公司,30,福建众邦药业有限公司,福建众邦药业有限公司,12,福建,福建,"[福建, 众邦, 药业, 有限公司]","[福建, 众邦, 药业, 有限公司]",福建 众邦 药业 有限公司,福建 众邦 药业 有限公司,100
460,000004.SZ,国华网安,福建摩格网络科技有限公司,福建摩格网络科技有限公司,31,福建乐游网络科技有限公司,福建乐游网络科技有限公司,11,福建,福建,"[福建, 摩格, 网络科技, 有限公司]","[福建, 乐游, 网络科技, 有限公司]",福建 乐游 网络科技 有限公司,福建 摩格 网络科技 有限公司,87
462,000004.SZ,国华网安,福建摩格网络科技有限公司,福建摩格网络科技有限公司,31,福建摩格网络科技有限公司,福建摩格网络科技有限公司,13,福建,福建,"[福建, 摩格, 网络科技, 有限公司]","[福建, 摩格, 网络科技, 有限公司]",福建 摩格 网络科技 有限公司,福建 摩格 网络科技 有限公司,100
551,000006.SZ,深振业A,天虹商场股份有限公司,天虹商场股份有限公司,2,天虹商场股份有限公司,天虹商场股份有限公司,3,None,None,"[天虹, 商场, 股份, 有限公司]","[天虹, 商场, 股份, 有限公司]",天虹 商场 股份 有限公司,天虹 商场 股份 有限公司,100
599,000006.SZ,深振业A,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,7,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,6,深圳,深圳,"[深圳市, 前海同, 创汇, 产业, 运营, 管理, 有限公司]","[深圳市, 前海同, 创汇, 产业, 运营, 管理, 有限公司]",深圳市 前海同 创汇 产业 运营 管理 有限公司,深圳市 前海同 创汇 产业 运营 管理 有限公司,100
619,000006.SZ,深振业A,深圳市南山区建筑工务署,深圳市南山区建筑工务署,9,深圳市南山区建筑工务署,深圳市南山区建筑工务署,8,深圳,深圳,"[深圳市, 南山区, 建筑, 工务, 署]","[深圳市, 南山区, 建筑, 工务, 署]",深圳市 南山区 建筑 工务 署,深圳市 南山区 建筑 工务 署,100
797,000007.SZ,全新好,MAPLE PACKING INDUSTRIAL (H.K.) LIMITED,MAPLE PACKING INDUSTRIAL (H.K.) LIMITED,5,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,"[MAPLE, , PACKING, , INDUSTRIAL, , (, H, .,...","[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...",maplepackingindustrialhklimited,maplepackingindustrialhklimited,100
814,000007.SZ,全新好,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,6,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,"[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...","[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...",maplepackingindustrialhklimited,maplepackingindustrialhklimited,100


In [80]:
output_path2 = os.path.join(root_dir, "df_fuzzymatch85_v2.csv")
df_fuzzymatch85.to_csv(output_path2, encoding = 'utf-8-sig')

#### (1) Choose 90 as a cutoff score

In [74]:
df_fuzzymatch90 = df_fuzzymatch[df_fuzzymatch['match_score'] >= 90]
df_fuzzymatch90.head(10)

,wind_code,sec_name,ar_by_buyer_name,ar_buyer_standard,ar_buyer_subid,cus_name,cus_name_standard,cus_name_subid,ar_buyer_region,cus_region,ar_buyer_seg,cus_name_seg,cus_name_seg_str,ar_buyer_seg_str,match_score
430,000004.SZ,国华网安,福建乐游网络科技有限公司,福建乐游网络科技有限公司,29,福建乐游网络科技有限公司,福建乐游网络科技有限公司,11,福建,福建,"[福建, 乐游, 网络科技, 有限公司]","[福建, 乐游, 网络科技, 有限公司]",福建 乐游 网络科技 有限公司,福建 乐游 网络科技 有限公司,100
446,000004.SZ,国华网安,福建众邦药业有限公司,福建众邦药业有限公司,30,福建众邦药业有限公司,福建众邦药业有限公司,12,福建,福建,"[福建, 众邦, 药业, 有限公司]","[福建, 众邦, 药业, 有限公司]",福建 众邦 药业 有限公司,福建 众邦 药业 有限公司,100
462,000004.SZ,国华网安,福建摩格网络科技有限公司,福建摩格网络科技有限公司,31,福建摩格网络科技有限公司,福建摩格网络科技有限公司,13,福建,福建,"[福建, 摩格, 网络科技, 有限公司]","[福建, 摩格, 网络科技, 有限公司]",福建 摩格 网络科技 有限公司,福建 摩格 网络科技 有限公司,100
551,000006.SZ,深振业A,天虹商场股份有限公司,天虹商场股份有限公司,2,天虹商场股份有限公司,天虹商场股份有限公司,3,None,None,"[天虹, 商场, 股份, 有限公司]","[天虹, 商场, 股份, 有限公司]",天虹 商场 股份 有限公司,天虹 商场 股份 有限公司,100
599,000006.SZ,深振业A,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,7,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,6,深圳,深圳,"[深圳市, 前海同, 创汇, 产业, 运营, 管理, 有限公司]","[深圳市, 前海同, 创汇, 产业, 运营, 管理, 有限公司]",深圳市 前海同 创汇 产业 运营 管理 有限公司,深圳市 前海同 创汇 产业 运营 管理 有限公司,100
619,000006.SZ,深振业A,深圳市南山区建筑工务署,深圳市南山区建筑工务署,9,深圳市南山区建筑工务署,深圳市南山区建筑工务署,8,深圳,深圳,"[深圳市, 南山区, 建筑, 工务, 署]","[深圳市, 南山区, 建筑, 工务, 署]",深圳市 南山区 建筑 工务 署,深圳市 南山区 建筑 工务 署,100
797,000007.SZ,全新好,MAPLE PACKING INDUSTRIAL (H.K.) LIMITED,MAPLE PACKING INDUSTRIAL (H.K.) LIMITED,5,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,"[MAPLE, , PACKING, , INDUSTRIAL, , (, H, .,...","[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...",maplepackingindustrialhklimited,maplepackingindustrialhklimited,100
814,000007.SZ,全新好,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,6,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,"[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...","[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...",maplepackingindustrialhklimited,maplepackingindustrialhklimited,100
831,000007.SZ,全新好,MAPLEPACKINGINDUSTRIALLIMITED,MAPLEPACKINGINDUSTRIALLIMITED,7,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,[MAPLEPACKINGINDUSTRIALLIMITED],"[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...",maplepackingindustrialhklimited,maplepackingindustriallimited,97
900,000007.SZ,全新好,WANDE INDUSTRIAL (HK) LIMITED,WANDE INDUSTRIAL (HK) LIMITED,11,WANDEINDUSTRIAL(HK)LIMITED,WANDEINDUSTRIAL(HK)LIMITED,2,None,None,"[WANDE, , INDUSTRIAL, , (, HK, ), , LIMITED]","[WANDEINDUSTRIAL, (, HK, ), LIMITED]",wandeindustrialhklimited,wandeindustrialhklimited,100


In [81]:
output_path3 = os.path.join(root_dir, "df_fuzzymatch90_v2.csv")
df_fuzzymatch90.to_csv(output_path3, encoding = 'utf-8-sig')